In [1]:
import pandas as pd

In [2]:
dataset = pd.read_csv('moviedataset.csv' , encoding='cp437')

In [3]:
dataset = dataset[['Title','Genre','Director','Actors','Plot']]
dataset.tail()

,Title,Genre,Director,Actors,Plot
245,The Lost Weekend,"Drama, Film-Noir",Billy Wilder,"Ray Milland, Jane Wyman, Phillip Terry, Howard...",The desperate life of a chronic alcoholic is f...
246,Short Term 12,Drama,Destin Daniel Cretton,"Brie Larson, John Gallagher Jr., Stephanie Bea...",A 20-something supervising staff member of a r...
247,His Girl Friday,"Comedy, Drama, Romance",Howard Hawks,"Cary Grant, Rosalind Russell, Ralph Bellamy, G...",A newspaper editor uses every trick in the boo...
248,The Straight Story,"Biography, Drama",David Lynch,"Sissy Spacek, Jane Galloway Heitz, Joseph A. C...",An old man makes a long journey by lawn-mover ...
249,Slumdog Millionaire,Drama,"Danny Boyle, Loveleen Tandan","Dev Patel, Saurabh Shukla, Anil Kapoor, Raj Zu...",A Mumbai teen reflects on his upbringing in th...


In [4]:
row=dataset['Genre'][1]+dataset['Director'][1]+dataset['Actors'][1]+dataset['Plot'][1]
row

'Crime, DramaFrancis Ford CoppolaMarlon Brando, Al Pacino, James Caan, Richard S. CastellanoThe aging patriarch of an organized crime dynasty transfers control of his clandestine empire to his reluctant son.'

In [7]:
import re
review = re.sub('[^a-zA-Z]',' ',row)
review

'Crime  DramaFrancis Ford CoppolaMarlon Brando  Al Pacino  James Caan  Richard S  CastellanoThe aging patriarch of an organized crime dynasty transfers control of his clandestine empire to his reluctant son '

In [10]:
review=review.lower()

In [12]:
review
review=review.split()

In [15]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
review=[ps.stem(word) for word in review]
print(review)

['crime', 'dramafr', 'ford', 'coppolamarlon', 'brando', 'al', 'pacino', 'jame', 'caan', 'richard', 's', 'castellanoth', 'age', 'patriarch', 'of', 'an', 'organ', 'crime', 'dynasti', 'transfer', 'control', 'of', 'hi', 'clandestin', 'empir', 'to', 'hi', 'reluct', 'son']


In [16]:
from nltk.corpus import stopwords

In [20]:
review=[ word for word in review if word not in set(stopwords.words('english'))]
print(review)

['crime', 'dramafr', 'ford', 'coppolamarlon', 'brando', 'al', 'pacino', 'jame', 'caan', 'richard', 'castellanoth', 'age', 'patriarch', 'organ', 'crime', 'dynasti', 'transfer', 'control', 'hi', 'clandestin', 'empir', 'hi', 'reluct', 'son']
